In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,471 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,275 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restricte

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
homeSales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
homeSales_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
# 2. Create a temporary view of the DataFrame.
homeSales_df.createOrReplaceTempView('homeSalesData')


In [5]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
four_bedroom_avg_price = spark.sql("SELECT YEAR(date), ROUND(AVG(price), 2)  FROM homeSalesData WHERE bedrooms == 4 GROUP BY YEAR(date)").show()


+----------+--------------------+
|year(date)|round(avg(price), 2)|
+----------+--------------------+
|      2022|           296363.88|
|      2019|            300263.7|
|      2020|           298353.78|
|      2021|           301819.44|
+----------+--------------------+



In [6]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
avg_3B_3B = spark.sql("SELECT ROUND(AVG(price),2) AS avg_price, date_built FROM homeSalesData WHERE bedrooms == 3 AND bathrooms == 3 GROUP BY date_built ORDER BY date_built").show()


+---------+----------+
|avg_price|date_built|
+---------+----------+
|292859.62|      2010|
|291117.47|      2011|
|293683.19|      2012|
|295962.27|      2013|
|290852.27|      2014|
| 288770.3|      2015|
|290555.07|      2016|
|292676.79|      2017|
+---------+----------+



In [7]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
avg_3B_3B_2F = spark.sql("SELECT ROUND(AVG(price),2) AS avg_price, date_built FROM homeSalesData WHERE bedrooms == 3 AND bathrooms == 3 AND floors == 2 AND sqft_living >= 2000 GROUP BY date_built ORDER BY date_built").show()


+---------+----------+
|avg_price|date_built|
+---------+----------+
|285010.22|      2010|
|276553.81|      2011|
|307539.97|      2012|
|303676.79|      2013|
|298264.72|      2014|
|297609.97|      2015|
| 293965.1|      2016|
|280317.58|      2017|
+---------+----------+



In [8]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

view_rating = spark.sql("SELECT view, ROUND(AVG(price),2) AS avg_price FROM homeSalesData GROUP BY view HAVING avg_price >= 350000 ").show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  51| 788128.21|
|  54| 798684.82|
|  69| 750537.94|
|  87| 1072285.2|
|  73| 752861.18|
|  64| 767036.67|
|  59|  791453.0|
|  85|1056336.74|
|  52| 733780.26|
|  71|  775651.1|
|  98|1053739.33|
|  99|1061201.42|
|  96|1017815.92|
| 100| 1026669.5|
|  70| 695865.58|
|  61| 746877.59|
|  75|1114042.94|
|  78|1080649.37|
|  89|1107839.15|
|  77|1076205.56|
+----+----------+
only showing top 20 rows

--- 2.1544992923736572 seconds ---


In [9]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table homeSalesData")

DataFrame[]

In [10]:
# 8. Check if the table is cached.
spark.catalog.isCached('homeSalesData')

True

In [11]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

view_rating = spark.sql("SELECT view, ROUND(AVG(price),2) AS avg_price FROM homeSalesData GROUP BY view HAVING avg_price >= 350000 ").show()


print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view| avg_price|
+----+----------+
|  51| 788128.21|
|  54| 798684.82|
|  69| 750537.94|
|  87| 1072285.2|
|  73| 752861.18|
|  64| 767036.67|
|  59|  791453.0|
|  85|1056336.74|
|  52| 733780.26|
|  71|  775651.1|
|  98|1053739.33|
|  99|1061201.42|
|  96|1017815.92|
| 100| 1026669.5|
|  70| 695865.58|
|  61| 746877.59|
|  75|1114042.94|
|  78|1080649.37|
|  89|1107839.15|
|  77|1076205.56|
+----+----------+
only showing top 20 rows

--- 1.1857950687408447 seconds ---


In [12]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
homeSales_df.write.parquet('home_partitioned', mode='overwrite')

In [13]:
# 11. Read the parquet formatted data.
partitioned_homeSales_df = spark.read.parquet('home_partitioned')

In [14]:
# 12. Create a temporary table for the parquet data.
partitioned_homeSales_df.createOrReplaceTempView("partitioned_homeSales_tempView")

In [15]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

view_rating = spark.sql("SELECT view, ROUND(AVG(price),2) AS avg_price FROM partitioned_homeSales_tempView GROUP BY view HAVING avg_price >= 350000 ").show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  51| 788128.21|
|  54| 798684.82|
|  69| 750537.94|
|  87| 1072285.2|
|  73| 752861.18|
|  64| 767036.67|
|  59|  791453.0|
|  85|1056336.74|
|  52| 733780.26|
|  71|  775651.1|
|  98|1053739.33|
|  99|1061201.42|
|  96|1017815.92|
| 100| 1026669.5|
|  70| 695865.58|
|  61| 746877.59|
|  75|1114042.94|
|  78|1080649.37|
|  89|1107839.15|
|  77|1076205.56|
+----+----------+
only showing top 20 rows

--- 0.8320071697235107 seconds ---


In [16]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table homeSalesData ")

DataFrame[]

In [17]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('homeSalesData')


False